In [1]:
import glob

import numpy as np
import pandas as pd

### Load dataset

In [2]:
parts = glob.glob('C:/ml_data/cic_iot_2023/raw/*.csv')
parts

['C:/ml_data/cic_iot_2023/raw\\part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv',
 'C:/ml_data/cic_iot_2023/raw\\part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv',
 'C:/ml_data/cic_iot_2023/raw\\part-00002-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv',
 'C:/ml_data/cic_iot_2023/raw\\part-00003-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv',
 'C:/ml_data/cic_iot_2023/raw\\part-00004-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv',
 'C:/ml_data/cic_iot_2023/raw\\part-00005-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv',
 'C:/ml_data/cic_iot_2023/raw\\part-00006-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv',
 'C:/ml_data/cic_iot_2023/raw\\part-00007-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv',
 'C:/ml_data/cic_iot_2023/raw\\part-00008-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv',
 'C:/ml_data/cic_iot_2023/raw\\part-00009-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv',
 'C:/ml_data/cic_iot_2023/raw\\part-00010-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv',
 'C:/ml_da

In [3]:
dfs = [pd.read_csv(filename) for filename in parts]
full_df = pd.concat(dfs, ignore_index=True)
full_df.head()

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000000,54.00,6.00,64.00,0.329807,0.329807,0.0,1.0,0.0,1.0,...,0.000000,54.00,8.334383e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-RSTFINFlood
1,0.000000,57.04,6.33,64.00,4.290556,4.290556,0.0,0.0,0.0,0.0,...,2.822973,57.04,8.292607e+07,9.5,10.464666,4.010353,160.987842,0.05,141.55,DoS-TCP_Flood
2,0.000000,0.00,1.00,64.00,33.396799,33.396799,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.312799e+07,9.5,9.165151,0.000000,0.000000,0.00,141.55,DDoS-ICMP_Flood
3,0.328175,76175.00,17.00,64.00,4642.133010,4642.133010,0.0,0.0,0.0,0.0,...,0.000000,50.00,8.301570e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,DoS-UDP_Flood
4,0.117320,101.73,6.11,65.91,6.202211,6.202211,0.0,0.0,1.0,0.0,...,23.113111,57.88,8.297300e+07,9.5,11.346876,32.716243,3016.808286,0.19,141.55,DoS-SYN_Flood


### Keep Dos and Benign

In [4]:
labels_to_keep = ["DoS-UDP_Flood", "DoS-TCP_Flood", "DoS-SYN_Flood", "BenignTraffic"]
df = full_df[full_df["label"].isin(labels_to_keep)]
df.reset_index(drop=True, inplace=True)

del full_df

In [5]:
df['label'].value_counts()

label
DoS-UDP_Flood    386668
DoS-TCP_Flood    311052
DoS-SYN_Flood    235060
BenignTraffic    127937
Name: count, dtype: int64

### Convert "Protocol Type" from number to str

In [6]:
iana_map = { 
    "0": "HOPOPT", "1": "ICMP", "2": "IGMP", "3": "GGP", "4": "IPv4", "5": "ST", 
    "6": "TCP", "7": "CBT", "8": "EGP", "9": "IGP", "10": "BBN-RCC-MON", "11": "NVP-II", 
    "12": "PUP", "13": "ARGUS (deprecated)", "14": "EMCON", "15": "XNET", "16": "CHAOS", 
    "17": "UDP", "18": "MUX", "19": "DCN-MEAS", "20": "HMP", "21": "PRM", "22": "XNS-IDP", 
    "23": "TRUNK-1", "24": "TRUNK-2", "25": "LEAF-1", "26": "LEAF-2", "27": "RDP", 
    "28": "IRTP", "29": "ISO-TP4", "30": "NETBLT", "31": "MFE-NSP", "32": "MERIT-INP", 
    "33": "DCCP", "34": "3PC", "35": "IDPR", "36": "XTP", "37": "DDP", "38": "IDPR-CMTP", 
    "39": "TP++", "40": "IL", "41": "IPv6", "42": "SDRP", "43": "IPv6-Route", 
    "44": "IPv6-Frag", "45": "IDRP", "46": "RSVP", "47": "GRE", "48": "DSR", "49": "BNA", 
    "50": "ESP", "51": "AH", "52": "I-NLSP", "53": "SWIPE (deprecated)", "54": "NARP", 
    "55": "MOBILE", "56": "TLSP", "57": "SKIP", "58": "IPv6-ICMP", "59": "IPv6-NoNxt", 
    "60": "IPv6-Opts", "62": "CFTP", "64": "SAT-EXPAK", "65": "KRYPTOLAN", "66": "RVD", 
    "67": "IPPC", "69": "SAT-MON", "70": "VISA", "71": "IPCV", "72": "CPNX", "73": "CPHB", 
    "74": "WSN", "75": "PVP", "76": "BR-SAT-MON", "77": "SUN-ND", "78": "WB-MON", 
    "79": "WB-EXPAK", "80": "ISO-IP", "81": "VMTP", "82": "SECURE-VMTP", "83": "VINES", 
    "84": "IPTM", "85": "NSFNET-IGP", "86": "DGP", "87": "TCF", "88": "EIGRP", 
    "89": "OSPFIGP", "90": "Sprite-RPC", "91": "LARP", "92": "MTP", "93": "AX.25", 
    "94": "IPIP", "95": "MICP (deprecated)","96": "SCC-SP", "97": "ETHERIP", "98": "ENCAP", 
    "100": "GMTP", "101": "IFMP", "102": "PNNI", "103": "PIM", "104": "ARIS", "105": "SCPS", 
    "106": "QNX", "107": "A/N", "108": "IPComp", "109": "SNP", "110": "Compaq-Peer", 
    "111": "IPX-in-IP", "112": "VRRP", "113": "PGM", "114": "", "115": "L2TP", "116": "DDX",  
    "117": "IATP", "118": "STP", "119": "SRP", "120": "UTI", "121": "SMP", 
    "122": "SM (deprecated)", "123": "PTP","124": "ISIS over IPv4", "125": "FIRE", 
    "126": "CRTP", "127": "CRUDP", "128": "SSCOPMCE", "129": "IPLT", "130": "SPS", 
    "131": "PIPE", "132": "SCTP",  "133": "FC", "134": "RSVP-E2E-IGNORE", 
    "135": "Mobility Header", "136": "UDPLite", "137": "MPLS-in-IP", "138": "manet", 
    "139": "HIP", "140": "Shim6", "141": "WESP", "142": "ROHC", "143": "Ethernet", 
    "144": "AGGFRAG", "145": "NSH"
}

df["Protocol Type"] = df["Protocol Type"].apply(lambda num : iana_map[ str(int(num)) ])

In [7]:
df["Protocol Type"].value_counts()

Protocol Type
TCP                   569643
UDP                   233095
CHAOS                 132571
ST                     37703
CBT                    24424
EGP                    13881
XNET                   11236
IGP                     9056
BBN-RCC-MON             6321
EMCON                   5847
ARGUS (deprecated)      4812
NVP-II                  4725
PUP                     4456
IPv4                    2235
GGP                      270
ICMP                     248
IGMP                     173
HOPOPT                    21
Name: count, dtype: int64

### Encode different types of Dos to one label

In [8]:
reduced_labels = {
    'DDoS-RSTFINFlood': 'DDoS', 'DDoS-PSHACK_Flood': 'DDoS', 'DDoS-SYN_Flood': 'DDoS', 
    'DDoS-UDP_Flood': 'DDoS', 'DDoS-TCP_Flood': 'DDoS', 'DDoS-ICMP_Flood': 'DDoS', 
    'DDoS-SynonymousIP_Flood': 'DDoS', 'DDoS-ACK_Fragmentation': 'DDoS', 
    'DDoS-UDP_Fragmentation': 'DDoS', 'DDoS-ICMP_Fragmentation': 'DDoS', 
    'DDoS-SlowLoris': 'DDoS', 'DDoS-HTTP_Flood': 'DDoS', 'DoS-UDP_Flood': 'DoS', 
    'DoS-SYN_Flood': 'DoS', 'DoS-TCP_Flood': 'DoS', 'DoS-HTTP_Flood': 'DoS', 
    'Mirai-greeth_flood': 'Mirai', 'Mirai-greip_flood': 'Mirai', 'Mirai-udpplain': 'Mirai', 
    'Recon-PingSweep': 'Recon', 'Recon-OSScan': 'Recon', 'Recon-PortScan': 'Recon', 
    'VulnerabilityScan': 'Recon', 'Recon-HostDiscovery': 'Recon', 'DNS_Spoofing': 'Spoofing', 
    'MITM-ArpSpoofing': 'Spoofing', 'BenignTraffic': 'Benign', 'BrowserHijacking': 'Web', 
    'Backdoor_Malware': 'Web', 'XSS': 'Web', 'Uploading_Attack': 'Web', 'SqlInjection': 'Web', 
    'CommandInjection': 'Web', 'DictionaryBruteForce': 'BruteForce'
}

df['label'] = df['label'].apply(lambda attack_name : reduced_labels[attack_name])

In [9]:
df["label"].value_counts()

label
DoS       932780
Benign    127937
Name: count, dtype: int64

### Sample dataset

In [17]:
df1 = df.sample(n=100000, ignore_index=True)
df2 = df.sample(n=400000, ignore_index=True)

In [18]:
df1["label"].value_counts()

label
DoS       88052
Benign    11948
Name: count, dtype: int64

In [19]:
df2["label"].value_counts()

label
DoS       351900
Benign     48100
Name: count, dtype: int64

### Save Dos and Benign before preprocessing

In [20]:
savepath1 = 'C:/ml_data/cic_iot_2023/cic_iot_2023_dos_benign_100000.csv'
savepath2 = 'C:/ml_data/cic_iot_2023/cic_iot_2023_dos_benign_400000.csv'
df1.to_csv(savepath1, index=False)
df2.to_csv(savepath2, index=False)